In [ ]:
pip install botorch

In [10]:
import pandas as pd
import torch

#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

train_data = pd.read_excel('data_sample_multiple_components_GLU_9objs.xlsx')
#print(train_data)
train_x = train_data.iloc[:,[0,1]].values
#print(train_X)
train_x = torch.tensor(train_x, dtype=torch.float32)
print(train_x)

train_obj = train_data.iloc[:,2:].values
#print(train_Y)
train_obj = torch.tensor(train_obj, dtype=torch.float32)
print(train_obj)

tensor([[0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000]])
tensor([[ 0.0450,  0.0500,  0.0090,  0.0190,  0.1520,  0.0000,  0.0210,  0.0270,
         -0.1990],
        [ 0.0390,  0.0540,  0.0130,  0.0640,  0.1800,  0.0000,  0.0260,  0.0210,
         -0.1990],
        [ 0.0470,  0.0660,  0.0140,  0.0280,  0.2280,  0.0000,  0.0330,  0.0400,
         -0.1990],
        [ 0.0430,  0.0570,  0.0120,  0.0370,  0.1870,  0.0000,  0.0270,  0.

In [21]:
from botorch import fit_gpytorch_model
from botorch.models.gp_regression import FixedNoiseGP
from botorch.models.transforms.outcome import Standardize
from botorch.models.model_list_gp_regression import ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize
from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.optim.optimize import optimize_acqf

torch.manual_seed(0)

NOISE_SE = torch.tensor([1,1,1,1,1,1,1,1,1])
#print(NOISE_SE)

# define models for objective and constraint
models = []
for i in range(train_obj.shape[-1]):
    train_y = train_obj[..., i:i+1]
    train_yvar = torch.full_like(train_y, NOISE_SE[i] ** 2)
    models.append(FixedNoiseGP(train_x, train_y, train_yvar, outcome_transform=Standardize(m=1)))
model = ModelListGP(*models)
mll = SumMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_model(mll)

BATCH_SIZE = 10
NUM_RESTARTS = 2
RAW_SAMPLES = 4
MC_SAMPLES = 16

standard_bounds = torch.tensor([[0.001, 0.1],[0.05, 3]])
#print(standard_bounds)
ref_point=torch.tensor([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0])

with torch.no_grad():
  #pred = model.posterior(normalize(train_x, standard_bounds)).mean
  pred = model.posterior(train_x).mean
partitioning = FastNondominatedPartitioning(ref_point, Y=pred)
sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)
acq_func = qExpectedHypervolumeImprovement(model,ref_point,partitioning,sampler)

candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        #options={"batch_limit": 5, "maxiter": 200},
        sequential=True)
#candidates =  unnormalize(candidates.detach(), bounds=standard_bounds)
print(candidates)

tensor([[1.8542e-02, 8.3327e-01],
        [1.3619e-02, 2.5995e+00],
        [1.9970e-02, 2.2111e-01],
        [4.4772e-02, 2.0989e+00],
        [3.1184e-02, 1.0067e+00],
        [1.2088e-03, 3.8620e-01],
        [4.0980e-03, 6.6677e-01],
        [4.7046e-02, 1.7213e+00],
        [3.3898e-02, 2.4039e+00],
        [9.6460e-03, 2.4084e+00]])


In [39]:
for i in range(9):
    print(model.models[i].covar_module.outputscale, 
          model.models[i].covar_module.base_kernel.lengthscale, 
          model.models[i].likelihood.noise)

tensor(6.6369, grad_fn=<SoftplusBackward0>) tensor([[0.3329, 0.3337]], grad_fn=<SoftplusBackward0>) tensor([23962.4922, 23962.4922, 23962.4922, 23962.4922, 23962.4922, 23962.4922,
        23962.4922, 23962.4922, 23962.4922, 23962.4922, 23962.4922, 23962.4922,
        23962.4922, 23962.4922, 23962.4922, 23962.4922, 23962.4922, 23962.4922,
        23962.4922, 23962.4922, 23962.4922, 23962.4922, 23962.4922, 23962.4922])
tensor(6.6329, grad_fn=<SoftplusBackward0>) tensor([[0.3339, 0.3332]], grad_fn=<SoftplusBackward0>) tensor([11423.8389, 11423.8389, 11423.8389, 11423.8389, 11423.8389, 11423.8389,
        11423.8389, 11423.8389, 11423.8389, 11423.8389, 11423.8389, 11423.8389,
        11423.8389, 11423.8389, 11423.8389, 11423.8389, 11423.8389, 11423.8389,
        11423.8389, 11423.8389, 11423.8389, 11423.8389, 11423.8389, 11423.8389])
tensor(6.6523, grad_fn=<SoftplusBackward0>) tensor([[0.3333, 0.3333]], grad_fn=<SoftplusBackward0>) tensor([82449.5938, 82449.5938, 82449.5938, 82449.5938, 82